### ITEM-ITEM 協同過濾：找出與目前瀏覽的商品最相似的商品族群(ITEM-ITEM Similarity Matrix)，推薦給顧客。
- https://towardsdatascience.com/recommender-systems-item-customer-collaborative-filtering-ff0c8f41ae8a

In [1]:
import pandas as pd
df = pd.read_csv('../transaction_data_version3.csv')
product_df = pd.read_csv('../product.csv')
product_df1 = product_df[['PRODUCT_ID','SUB_COMMODITY_DESC']]
df_merge = df.merge(product_df1,on='PRODUCT_ID')
df_merge

,household_key,MEMBER,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,WEEK_NO,DAY,DATE,hour_time,SUB_COMMODITY_DESC
0,2375,0,26984851472,1004906,1,1.39,1,1,2019-07-30,16:00:00,POTATOES RUSSET (BULK&BAG)
1,1929,1,27021022215,1004906,1,1.39,1,4,2019-08-02,17:00:00,POTATOES RUSSET (BULK&BAG)
2,568,0,27021090189,1004906,1,1.39,1,4,2019-08-02,00:00:00,POTATOES RUSSET (BULK&BAG)
3,887,0,27031467254,1004906,1,1.39,1,5,2019-08-03,15:00:00,POTATOES RUSSET (BULK&BAG)
4,175,0,27057508267,1004906,1,1.99,2,7,2019-08-05,23:00:00,POTATOES RUSSET (BULK&BAG)
...,...,...,...,...,...,...,...,...,...,...,...
2551702,1598,0,42305362535,92130,1,0.99,102,711,2021-07-09,15:00:00,MARGARINE: TUBS AND BOWLS
2551703,1598,0,42305362535,114102,1,8.89,102,711,2021-07-09,15:00:00,BEERALEMALT LIQUORS
2551704,1598,0,42305362535,133449,1,6.99,102,711,2021-07-09,15:00:00,BEERALEMALT LIQUORS
2551705,1598,0,42305362535,6923644,1,4.50,102,711,2021-07-09,15:00:00,BEERALEMALT LIQUORS


In [2]:
df = df_merge.copy()
df.head()

,household_key,MEMBER,BASKET_ID,PRODUCT_ID,QUANTITY,SALES_VALUE,WEEK_NO,DAY,DATE,hour_time,SUB_COMMODITY_DESC
0,2375,0,26984851472,1004906,1,1.39,1,1,2019-07-30,16:00:00,POTATOES RUSSET (BULK&BAG)
1,1929,1,27021022215,1004906,1,1.39,1,4,2019-08-02,17:00:00,POTATOES RUSSET (BULK&BAG)
2,568,0,27021090189,1004906,1,1.39,1,4,2019-08-02,00:00:00,POTATOES RUSSET (BULK&BAG)
3,887,0,27031467254,1004906,1,1.39,1,5,2019-08-03,15:00:00,POTATOES RUSSET (BULK&BAG)
4,175,0,27057508267,1004906,1,1.99,2,7,2019-08-05,23:00:00,POTATOES RUSSET (BULK&BAG)


In [3]:
df_new = df[['household_key','SUB_COMMODITY_DESC','QUANTITY']]

In [4]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551707 entries, 0 to 2551706
Data columns (total 3 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   household_key       int64 
 1   SUB_COMMODITY_DESC  object
 2   QUANTITY            int64 
dtypes: int64(2), object(1)
memory usage: 77.9+ MB


In [5]:
df_new.head()

,household_key,SUB_COMMODITY_DESC,QUANTITY
0,2375,POTATOES RUSSET (BULK&BAG),1
1,1929,POTATOES RUSSET (BULK&BAG),1
2,568,POTATOES RUSSET (BULK&BAG),1
3,887,POTATOES RUSSET (BULK&BAG),1
4,175,POTATOES RUSSET (BULK&BAG),1


In [6]:
df_group = df_new.groupby(['household_key','SUB_COMMODITY_DESC']).sum().reset_index()
df_group.head()

,household_key,SUB_COMMODITY_DESC,QUANTITY
0,1,ADULT ANALGESICS,3
1,1,ADULT CEREAL,2
2,1,AEROSOL TOPPINGS,1
3,1,AIR CARE - AEROSOLS,1
4,1,AIR CARE - CANDLES,6


In [7]:
#import numpy as np 
#customers = list(np.sort(df_group.household_key.unique()))
## a list of values, so customers now stores 36 unique customers
#products = list(df_group.SUB_COMMODITY_DESC.unique()) 
#quantity = list(df_group.QUANTITY)

In [8]:
#DfCustomerUnique = pd.DataFrame(customers,columns=['Customer']) 
#DfCustomerUnique.head()

In [9]:
#from scipy import sparse 
#from pandas.api.types import CategoricalDtype 
#
#rows = df_group.household_key.astype(CategoricalDtype(categories=customers)).cat.codes 
## We have got 35 unique customers, which make up 13837 data rows (index) 
## Get the associated row indices 
#
#cols = df_group.SUB_COMMODITY_DESC.astype(CategoricalDtype(categories= products)).cat.codes 
## We have got unique 3725 SalesItems, making up 13837 data rows (index) 
## Get the associated column indices 
##Compressed Sparse Row matrix 
#
#PurchaseSparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products))) 
##len of customers=35, len of products=3725 
##csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)]) 
##where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]. , see [3]
#
#PurchaseSparse

In [10]:
#MatrixSize = PurchaseSparse.shape[0]*PurchaseSparse.shape[1] # 130375 possible interactions in the matrix (35 unique customers * 3725 unique SalesItems=130375)
#PurchaseAmount = len(PurchaseSparse.nonzero()[0]) # 13837 SalesItems interacted with;
#sparsity = 100*(1 - (PurchaseAmount/MatrixSize))
#sparsity

In [11]:
def create_DataBinary(DataGrouped): 
    DataBinary = df_group.copy()
    DataBinary['PurchasedYes'] = 1 
    return DataBinary 
DataBinary = create_DataBinary(df_group) 
DataBinary

,household_key,SUB_COMMODITY_DESC,QUANTITY,PurchasedYes
0,1,ADULT ANALGESICS,3,1
1,1,ADULT CEREAL,2,1
2,1,AEROSOL TOPPINGS,1,1
3,1,AIR CARE - AEROSOLS,1,1
4,1,AIR CARE - CANDLES,6,1
...,...,...,...,...
608297,2500,WRITING INSTRUMENTS,2,1
608298,2500,YELLOW SUMMER SQUASH,1,1
608299,2500,YOGURT,6,1
608300,2500,YOGURT MULTI-PACKS,6,1


In [12]:
DataBinary.describe()

,household_key,QUANTITY,PurchasedYes
count,608302.000000,608302.000000,608302.0
mean,1267.778751,5.503431,1.0
std,723.302392,14.431976,0.0
min,1.000000,1.000000,1.0
25%,648.000000,1.000000,1.0
50%,1266.000000,2.000000,1.0
75%,1905.000000,5.000000,1.0
max,2500.000000,1888.000000,1.0


In [13]:
purchase_data=DataBinary.drop(['QUANTITY'], axis=1) 

In [14]:
purchase_data

,household_key,SUB_COMMODITY_DESC,PurchasedYes
0,1,ADULT ANALGESICS,1
1,1,ADULT CEREAL,1
2,1,AEROSOL TOPPINGS,1
3,1,AIR CARE - AEROSOLS,1
4,1,AIR CARE - CANDLES,1
...,...,...,...
608297,2500,WRITING INSTRUMENTS,1
608298,2500,YELLOW SUMMER SQUASH,1
608299,2500,YOGURT,1
608300,2500,YOGURT MULTI-PACKS,1


In [15]:
purchase_data['SUB_COMMODITY_DESC'] = purchase_data['SUB_COMMODITY_DESC'].astype(str) 
DfMatrix = pd.pivot_table(purchase_data, values='PurchasedYes', index='household_key', columns='SUB_COMMODITY_DESC')
DfMatrix.head()

SUB_COMMODITY_DESC,*ATH ACCES:TOWEL BARS/SOAP D,*ATTERIES:CAMERA/FLASH/WATCH,*BOYS/GIRLS MISC TOYS,*MISC. LOBBY ITEMS,*PURSES UMBRELLAS,*SCRAPBOOK,*SPORT NOVELTIES,*SPORTS APPAREL,*SPRING/SUMMER MISC,20 BIKES,...,WRITING INSTRUMENTS,XMAS PLUSH,YARDLEY,YEAST: DRY,YELLOW JACKET,YELLOW SUMMER SQUASH,YNG MEN SCREEN PRINT T-SHIRTS,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
household_key,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Normalization reduces the scale of the data to be in a range from 0 to 1:
- Xnormalized = X−Xmin / (Xmax−Xmin)

In [16]:
DfMatrix=DfMatrix.fillna(0) 
#NaN values need to get replaced by 0, meaning they have not been purchased yet.
DfMatrixNorm3 = (DfMatrix-DfMatrix.min())/(DfMatrix.max()-DfMatrix.min())
DfMatrixNorm3.head()

SUB_COMMODITY_DESC,*ATH ACCES:TOWEL BARS/SOAP D,*ATTERIES:CAMERA/FLASH/WATCH,*BOYS/GIRLS MISC TOYS,*MISC. LOBBY ITEMS,*PURSES UMBRELLAS,*SCRAPBOOK,*SPORT NOVELTIES,*SPORTS APPAREL,*SPRING/SUMMER MISC,20 BIKES,...,WRITING INSTRUMENTS,XMAS PLUSH,YARDLEY,YEAST: DRY,YELLOW JACKET,YELLOW SUMMER SQUASH,YNG MEN SCREEN PRINT T-SHIRTS,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
household_key,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
DfResetted = DfMatrix.reset_index().rename_axis(None, axis=1) 
DfResetted.head() 
#Now each row represents one customer`s buying behaviour: 1 means the customer has purchased, NaN the customer has not yet purchased it

,household_key,*ATH ACCES:TOWEL BARS/SOAP D,*ATTERIES:CAMERA/FLASH/WATCH,*BOYS/GIRLS MISC TOYS,*MISC. LOBBY ITEMS,*PURSES UMBRELLAS,*SCRAPBOOK,*SPORT NOVELTIES,*SPORTS APPAREL,*SPRING/SUMMER MISC,...,WRITING INSTRUMENTS,XMAS PLUSH,YARDLEY,YEAST: DRY,YELLOW JACKET,YELLOW SUMMER SQUASH,YNG MEN SCREEN PRINT T-SHIRTS,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df=DfResetted 

In [19]:
df=df.fillna(0) 
df.head()

,household_key,*ATH ACCES:TOWEL BARS/SOAP D,*ATTERIES:CAMERA/FLASH/WATCH,*BOYS/GIRLS MISC TOYS,*MISC. LOBBY ITEMS,*PURSES UMBRELLAS,*SCRAPBOOK,*SPORT NOVELTIES,*SPORTS APPAREL,*SPRING/SUMMER MISC,...,WRITING INSTRUMENTS,XMAS PLUSH,YARDLEY,YEAST: DRY,YELLOW JACKET,YELLOW SUMMER SQUASH,YNG MEN SCREEN PRINT T-SHIRTS,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
DfSalesItem = df.drop('household_key', axis=1) 
DfSalesItem.head()

,*ATH ACCES:TOWEL BARS/SOAP D,*ATTERIES:CAMERA/FLASH/WATCH,*BOYS/GIRLS MISC TOYS,*MISC. LOBBY ITEMS,*PURSES UMBRELLAS,*SCRAPBOOK,*SPORT NOVELTIES,*SPORTS APPAREL,*SPRING/SUMMER MISC,20 BIKES,...,WRITING INSTRUMENTS,XMAS PLUSH,YARDLEY,YEAST: DRY,YELLOW JACKET,YELLOW SUMMER SQUASH,YNG MEN SCREEN PRINT T-SHIRTS,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### So we can now calculate the Item based recommendation. We will normalize dataframe for comparising reasons:

In [21]:
import numpy as np
DfSalesItemNorm = DfSalesItem/np.sqrt(np.square(DfSalesItem).sum(axis=0))   
DfSalesItemNorm.head()

,*ATH ACCES:TOWEL BARS/SOAP D,*ATTERIES:CAMERA/FLASH/WATCH,*BOYS/GIRLS MISC TOYS,*MISC. LOBBY ITEMS,*PURSES UMBRELLAS,*SCRAPBOOK,*SPORT NOVELTIES,*SPORTS APPAREL,*SPRING/SUMMER MISC,20 BIKES,...,WRITING INSTRUMENTS,XMAS PLUSH,YARDLEY,YEAST: DRY,YELLOW JACKET,YELLOW SUMMER SQUASH,YNG MEN SCREEN PRINT T-SHIRTS,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.044455,0.0,0.0,0.0,0.0,0.065233,0.0,0.0,0.030345,0.025166
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.030345,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.044455,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


### Calculating with .dot Vectors to compute Cosine Similarities

In [22]:
ItemItemSim = DfSalesItemNorm.transpose().dot(DfSalesItemNorm) 
ItemItemSim.head()

,*ATH ACCES:TOWEL BARS/SOAP D,*ATTERIES:CAMERA/FLASH/WATCH,*BOYS/GIRLS MISC TOYS,*MISC. LOBBY ITEMS,*PURSES UMBRELLAS,*SCRAPBOOK,*SPORT NOVELTIES,*SPORTS APPAREL,*SPRING/SUMMER MISC,20 BIKES,...,WRITING INSTRUMENTS,XMAS PLUSH,YARDLEY,YEAST: DRY,YELLOW JACKET,YELLOW SUMMER SQUASH,YNG MEN SCREEN PRINT T-SHIRTS,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
*ATH ACCES:TOWEL BARS/SOAP D,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.101015,0.0,0.000000,0.0,0.065233,0.0,0.000000,0.000000,0.025166
*ATTERIES:CAMERA/FLASH/WATCH,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.052852,0.0,0.000000,0.0,0.000000,0.000000,0.035590
*BOYS/GIRLS MISC TOYS,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.068870,0.052164,0.0,0.019299,0.0,0.033686,0.0,0.000000,0.054845,0.064978
*MISC. LOBBY ITEMS,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.022228,0.050508,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.015172,0.025166
*PURSES UMBRELLAS,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.051333,0.058321,0.0,0.043153,0.0,0.018831,0.0,0.024311,0.078838,0.079912


In [23]:
ItemNeighbours = pd.DataFrame(index=ItemItemSim.columns,columns=range(1,12))
ItemNeighbours.head()

,1,2,3,4,5,6,7,8,9,10,11
*ATH ACCES:TOWEL BARS/SOAP D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
*ATTERIES:CAMERA/FLASH/WATCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
*BOYS/GIRLS MISC TOYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
*MISC. LOBBY ITEMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
*PURSES UMBRELLAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
for i in range(0,len(ItemItemSim.columns)): 
    ItemNeighbours.iloc[i,:11] = ItemItemSim.iloc[0:,i].sort_values(ascending=False)[:11].index 
    #we only have 9 items, so we can max recommend 9 items (itself included) 

ItemNeighbours

,1,2,3,4,5,6,7,8,9,10,11
*ATH ACCES:TOWEL BARS/SOAP D,*ATH ACCES:TOWEL BARS/SOAP D,UPDATED WOVENS,PATIO FURNITURE,BOOKCASES,SEAFOOD-FRE-SMOKED,SEAFOOD-MISC-RW-ALL,EXERCISE ACCESSORIES,LOIN - WH/HALF,JUMBO SHRUBS,FLOURSCENT LAMPS,GERBS
*ATTERIES:CAMERA/FLASH/WATCH,*ATTERIES:CAMERA/FLASH/WATCH,HAMS-CANNED,PEYTON/GM EXPENSE ITEMS,RX MEDICATED OTC DRUGS,DELI TRAY:CHICKEN,PREMIUM MUMS,BABY FURNITURE ACCESSORIES,SW GDS:SW ROLLS/DAN-LESS THN 6,BABY BASICS,BRIDAL-MAGAZINE,CLOSET ORGNZRS: HNGRS/STORER
*BOYS/GIRLS MISC TOYS,*BOYS/GIRLS MISC TOYS,GIRLS HOSIERY,SUNGLASSES KIDS,MISSY TRAD KNIT TOPS,DOOR CHARMS,TROPICALS,GARLAND,PREMIUM SHRUBS,VASES&CONTAINERS,SUNLESS PRODUCTS,HERBICIDES & WEED KILLERS
*MISC. LOBBY ITEMS,*MISC. LOBBY ITEMS,RIDE-ONS,BAKING,COTY FRAGRANCES,PRESCHOOL TOYS,TRADITIONAL RUM,OUTDOOR SPORTS,COLOR SETS,NERF ITEMS,AEROSOL DEODORANTS,OVEN CLEANERS
*PURSES UMBRELLAS,*PURSES UMBRELLAS,COVER PADS,EYE CENTER,PERSONAL CD,HEADCLEANERS,UMBRELLAS,DIET BARS DIET LIQUID MEALS,WATCHES WATCH BANDS,BATH OILS,MENS RUBBERWEAR,BATH RUGS
...,...,...,...,...,...,...,...,...,...,...,...
YELLOW SUMMER SQUASH,YELLOW SUMMER SQUASH,SQUASH ZUCCHINI,VEGETABLES ALL OTHER,ASPARAGUS,HARD SQUASH,ONIONS RED (BULK&BAG),GARLIC WHOLE CLOVES,OLIVE OIL,CHEESE:SPECIALTY PREPACK,LEMONS,DRIED FRUIT - OTHER
YNG MEN SCREEN PRINT T-SHIRTS,YNG MEN SCREEN PRINT T-SHIRTS,KIDS STORAGE,MENS GIFTS,DAFFODILS,GERMAN FOODS,SCIENTIFIC CALC,CONVALESCENT CARE - CANES,POLY POOLS,BULK PLASTICS,SMOKED CHOPS BONELESS,BREAST - BONE-IN (FRZ)
YOGURT,YOGURT,FLUID MILK,EGGS,CRACKERS,CANS SOUP/CHILI,COOKIES/SWEET GOODS,JUICE,FROZEN PIZZA,CEREAL - COLD,FROZEN BREAKFAST,ORGANIC CARROTS
YOGURT MULTI-PACKS,YOGURT MULTI-PACKS,BANANAS,YOGURT NOT MULTI-PACKS,SHREDDED CHEESE,PREMIUM,TORTILLA/NACHO CHIPS,FLUID MILK WHITE ONLY,POURABLE SALAD DRESSINGS,ALL FAMILY CEREAL,POTATO CHIPS,SOFT DRINKS 12/18&15PK CAN CAR


In [29]:
ItemNeighbours.iloc[:,1:11]

,2,3,4,5,6,7,8,9,10,11
*ATH ACCES:TOWEL BARS/SOAP D,UPDATED WOVENS,PATIO FURNITURE,BOOKCASES,SEAFOOD-FRE-SMOKED,SEAFOOD-MISC-RW-ALL,EXERCISE ACCESSORIES,LOIN - WH/HALF,JUMBO SHRUBS,FLOURSCENT LAMPS,GERBS
*ATTERIES:CAMERA/FLASH/WATCH,HAMS-CANNED,PEYTON/GM EXPENSE ITEMS,RX MEDICATED OTC DRUGS,DELI TRAY:CHICKEN,PREMIUM MUMS,BABY FURNITURE ACCESSORIES,SW GDS:SW ROLLS/DAN-LESS THN 6,BABY BASICS,BRIDAL-MAGAZINE,CLOSET ORGNZRS: HNGRS/STORER
*BOYS/GIRLS MISC TOYS,GIRLS HOSIERY,SUNGLASSES KIDS,MISSY TRAD KNIT TOPS,DOOR CHARMS,TROPICALS,GARLAND,PREMIUM SHRUBS,VASES&CONTAINERS,SUNLESS PRODUCTS,HERBICIDES & WEED KILLERS
*MISC. LOBBY ITEMS,RIDE-ONS,BAKING,COTY FRAGRANCES,PRESCHOOL TOYS,TRADITIONAL RUM,OUTDOOR SPORTS,COLOR SETS,NERF ITEMS,AEROSOL DEODORANTS,OVEN CLEANERS
*PURSES UMBRELLAS,COVER PADS,EYE CENTER,PERSONAL CD,HEADCLEANERS,UMBRELLAS,DIET BARS DIET LIQUID MEALS,WATCHES WATCH BANDS,BATH OILS,MENS RUBBERWEAR,BATH RUGS
...,...,...,...,...,...,...,...,...,...,...
YELLOW SUMMER SQUASH,SQUASH ZUCCHINI,VEGETABLES ALL OTHER,ASPARAGUS,HARD SQUASH,ONIONS RED (BULK&BAG),GARLIC WHOLE CLOVES,OLIVE OIL,CHEESE:SPECIALTY PREPACK,LEMONS,DRIED FRUIT - OTHER
YNG MEN SCREEN PRINT T-SHIRTS,KIDS STORAGE,MENS GIFTS,DAFFODILS,GERMAN FOODS,SCIENTIFIC CALC,CONVALESCENT CARE - CANES,POLY POOLS,BULK PLASTICS,SMOKED CHOPS BONELESS,BREAST - BONE-IN (FRZ)
YOGURT,FLUID MILK,EGGS,CRACKERS,CANS SOUP/CHILI,COOKIES/SWEET GOODS,JUICE,FROZEN PIZZA,CEREAL - COLD,FROZEN BREAKFAST,ORGANIC CARROTS
YOGURT MULTI-PACKS,BANANAS,YOGURT NOT MULTI-PACKS,SHREDDED CHEESE,PREMIUM,TORTILLA/NACHO CHIPS,FLUID MILK WHITE ONLY,POURABLE SALAD DRESSINGS,ALL FAMILY CEREAL,POTATO CHIPS,SOFT DRINKS 12/18&15PK CAN CAR


In [25]:
ItemNeighbours.iloc[1000]

1           HAND AND BODY CREAM
2     FACIAL SOAPS-SCRUBS-MASKS
3         HAND AND BODY LOTIONS
4                   LIQUID SOAP
5            ALKALINE BATTERIES
6      PLASTIC WRAP (SARANETC.)
7       TAPE & MAILING PRODUCTS
8                    TOOTHPASTE
9                       SHAMPOO
10      SPONGES: BATH HOUSEHOLD
11               CARDS EVERYDAY
Name: HAND AND BODY CREAM, dtype: object